In [1]:
import os
from argparse import ArgumentParser
from sklearn.externals import joblib
from tictacs import from_recipe
from pan import ProfilingDataset
import dill
import cPickle as pickle
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score, confusion_matrix


infolder = "../DATA/pan16-author-profiling-training-dataset-2016-04-25/pan16-author-profiling-training-dataset-english-2016-02-29/"
outfolder = "models/"
print('Loading dataset->Grouping User texts.\n')
dataset = ProfilingDataset(infolder)
print('Loaded {} users...\n'.format(len(dataset.entries)))
# get config
config = dataset.config
tasks = config.tasks
print('\n--------------- Thy time of Running ---------------')
for task in tasks:
    print('Learning to judge %s..' % task)
    # load data
    X, y = dataset.get_data(task)

Loading dataset->Grouping User texts.

Loaded 436 users...


--------------- Thy time of Running ---------------
Learning to judge age..
Learning to judge gender..


In [ ]:
Instance

In [17]:
from pan import ProfilingDataset, createDocProfiles, create_target_prof_trainset
from pan import preprocess

task = 'gender'
docs = createDocProfiles(dataset)
X, y = create_target_prof_trainset(docs, task)
print len(X)
#X = preprocess.preprocess(X)

277792


In [ ]:
Profile

In [68]:
#reload(preprocess)
#reload(features)
from pan import features
from pan import preprocess
#X, y = dataset.get_data('age')
X, y = dataset.get_data('gender')
print len(X)
#print X[0]
#X = preprocess.preprocess(X)
#print "$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$"
#print X[0]

436


In [ ]:
import pickle
with open("../DATA/graph-twitter/graph_gender_age/X.txt", 'r') as inp:
    X = pickle.load(inp)
with open("../DATA/graph-twitter/graph_gender_age/y.txt", 'r') as yinp:
    y = pickle.load(yinp)
X = preprocess.preprocess(X)
y = [s.lower() for s in y]

In [54]:
import json
with open("../DATA/graph-twitter/graph_gender_age/labeled.json", 'r') as inp:
    labeled = json.load(inp)
X = []
y = []
for user in labeled:
    y.append(user['gender'].lower())
    X.append(' '.join(user['texts']))
# for user in labeled:
#     y.extend([user['gender'].lower() for t in user['texts']])
#     X.extend(user['texts'])
print len(X), len(y)
#X = preprocess.preprocess(X)

303 303


In [ ]:
y

In [47]:
from sklearn.base import BaseEstimator, TransformerMixin

class SOACWrapper(BaseEstimator, TransformerMixin):
    
    import time
    import numpy
    from sklearn.preprocessing import normalize
    from collections import Counter
    
    
    def __init__(self):
        self.labels = []
        self.prior_row = None
        self.term_table = None
        return None
    
    def fit(self, X, y=None):
        
        import time
        import numpy
        from sklearn.preprocessing import normalize
        from collections import Counter
        
        if y is None:
            raise ValueError('we need y labels to supervise-fit!')
        else:
            time_start = time.time()
            target_profiles = sorted(list(set(y)))
            self.labels = target_profiles
            ##SOA
            doc_prof = numpy.zeros([X.shape[0], len(target_profiles)])
            for i in range(0, X.shape[0]):
                tmp = numpy.zeros([1, len(target_profiles)])
                tmp[0, target_profiles.index(y[i])] = 1
                doc_prof[i, :] = tmp
            
            ## SOAC
#            dd = Counter(y)
#             self.prior_row = numpy.zeros([1, len(target_profiles)])
#             for i, key in enumerate(sorted(dd.keys())):
#                 dd[key] = dd[key]/float(len(y))
#                 self.prior_row[0, i] = 1/dd[key]
#             doc_prof = numpy.tile(self.prior_row, (X.shape[0], 1))
#             for i in range(0, X.shape[0]):
#                 doc_prof[i, target_profiles.index(y[i])] = 0
#
            #print self.prior_row
            # Gia to palio. Na vgalw kai doc_prof apo to transform
            #self.prior_row = numpy.ones([1, len(target_profiles)])
            
            #try:
                #X = numpy.log2(X + 1)
            #except Exception, e:
            #    print "Error in log2"
            #    print e
            print 'X'
            print X.shape
            print X
            print 'Doc_prof'
            print doc_prof.shape
            print doc_prof
            try:
                term_prof = 100*X.transpose().dot(doc_prof)
            except Exception, e:
                print "Error in product"
                print e
            print 'Term_prof'
            print term_prof.shape
            print term_prof
#             print 'Dot Product'
#             print term_prof
            term_prof = term_prof / term_prof.sum(axis=0)
            print 'First'
            print term_prof
            #print "Normalization per collumn product in %0.2f sec" % (time.time()- time_start)
            #normalize(term_prof, norm='l1', axis=0, copy=False)
            # normalize across profiles
            print 'Second'
            term_prof = normalize(term_prof, norm='l1', axis=1, copy=False)
            #print term_prof
#             term_prof = term_prof / \
#                numpy.reshape(
#                   term_prof.sum(axis=1), (term_prof.sum(axis=1).shape[0], 1))
            print term_prof
            #print "Normalization per collumn product in %0.2f sec" % (time.time()- time_start)
#             print 'First'
#             term_prof = normalize(term_prof, norm='l1', axis=0, copy=False)
#             print term_prof
#             print 'Second'
#             term_prof = normalize(term_prof, norm='l1', axis=1, copy=False)
#             print term_prof
            # clean term_prof
            # term_prof = cleaned(term_prof, self.thres)
            self.term_table = term_prof
            return self
        
    def transform(self, X, y=None):
        
        import numpy
        from sklearn.preprocessing import normalize
        
        if self.labels is None:
            raise AttributeError('term_table was no found! \
                     Probably model was not fitted first. Run model.fit(X,y)!')
            exit(1)
        else:
            doc_prof = X.dot(self.term_table)
            for i in range(0, doc_prof.shape[0]):
                doc_prof[i, :] = doc_prof[i, :] #- doc_prof[i, :].min()
            # print doc_prof
            #normalize(doc_prof, norm='l1', axis=1, copy=False)
            return doc_prof
        
    def predict(self, X):
        
        import numpy
        num2label = dict((i, label) for i, label in enumerate(self.labels))
        Y = self.transform(X)
        y = numpy.argmax(Y, axis=1)
        final = numpy.array([num2label[i] for i in y])
#         for i,_ in enumerate(final):
#             print 'X'
#             print X[i,:]
#             print 'TR'
#             print Y[i,:]
#             print 'Max || Predict || Real'
#             print y[i], final[i], y_true[i]
        return final

        

In [ ]:
import numpy
from sklearn.preprocessing import normalize
a = numpy.random.rand(2,2)
print a
normalize(a, norm='l1', axis=1, copy=False)`
print a

In [ ]:
numpy.max(tr, axis=0)

In [ ]:
#test = X[11000]
c = CharFeatureExtractor(True)
#tr = c.transform(X_train)
soacw = SOACWrapper()
pipe3 = Pipeline([('CharCount', c), ('soac', soacw)])
#soacw.fit(tr, y_train)
gg = pipe3.fit_transform(X_train, y_train)

In [ ]:
pipe3.steps[1][1].term_table

In [ ]:
c = CharFeatureExtractor(True)
#tr = c.transform(X_train)
tr[4,:].dot(pipe3.steps[1][1].term_table)

In [ ]:
y_train[:10]

In [ ]:
predict = soacw.predict(c.transform(X_cv), y_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y_cv))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y_cv))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

In [ ]:
for i in index:
    print predict[i]

In [ ]:
index = []
for i in xrange(len(y_train)):
    if y_train[i] == '65-xx':
        print i
        index.append(i)
index.extend([9, 15, 28,40])
index

In [ ]:
gg[index]

In [ ]:
c2 = CharFeatureExtractor(True)
tr3 =c2.transform([X_train[i] for i in index])
for i in tr3:
    print tr3.argmax(), tr3.max()

In [35]:
import regex as re
import nltk
import numpy
from textblob.tokenizers import WordTokenizer
from sklearn.base import BaseEstimator, TransformerMixin
from pan.misc import _twokenize

class CharFeatureExtractor(BaseEstimator, TransformerMixin):

    """ Model that extracts Char Based Features. Counter: 32 features """

    def __init__(self, norm, norm_axis=0):
        self.letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j',
                  'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
                  'u', 'v', 'w', 'x', 'y', 'z']
        self.numbers = ['0', '1', '2', '3','4', '5', '6', '7', '8', '9']
        self.special_c = ['!', '.', ':', '?', ';', ',', ')', '(', 
                          '-', '%', '$', '#', '@', '^', '&', '*', 
                          '=', '+', '/', '"', "'", '<', '>', '|',
                          '~', '`']
        self.names = ['#C', '#a-z', '#upper', '#0-9', '#w', '#\t'] + ['No_'+str(i) for i in self.special_c]
        self.norm = norm
        self.norm_axis = norm_axis

    def fit(self, X, y=None):
        return self

    def transform(self, texts):
        """ transform data

        :texts: The texts to count capital words in
        :returns: array

        """
        from sklearn.preprocessing import normalize
        
        final_feat = []
        for text in texts:
            #text = text.encode('utf-8', 'ignore')
            tmp = []
            tmp.append(len(text))
            if tmp[0] == 0:
                tmp = [0 for i in self.names]
            else:
                tmp.append(sum([text.lower().count(l) for l in self.letters])/float(tmp[0]))
                tmp.append(sum([l.isupper() for l in text])/float(tmp[0]))
                tmp.append(sum([text.lower().count(l) for l in self.numbers])/float(tmp[0]))
                tmp.append(text.count(' ')/float(tmp[0]))
                tmp.append(text.count('\t')/float(tmp[0]))
                tmp.extend([text.lower().count(l)/float(tmp[0]) for l in self.special_c])
            final_feat.append(tmp)
        if self.norm:
            final_feat2 = normalize(numpy.array(final_feat), norm='l1', axis=0)
            #final_feat2 = normalize(numpy.array(final_feat), norm='l1', axis=1)
            return final_feat2
            #return normalize(numpy.array(final_feat), norm='l1', axis=self.norm_axis)
        else:
            return numpy.array(final_feat) 

In [73]:
import regex as re
import nltk
import numpy
from textblob.tokenizers import WordTokenizer
from sklearn.base import BaseEstimator, TransformerMixin
from pan.misc import _twokenize

eyes = [':', ';']
noses = ['', '-', '=', 'o', 'O', '0', '>']
mouths = [')', 'D', ']', '>',  '(',  '[', '<', 'p', 'P', 'd', "|", '/', '*', '@', '!', '$', '3', 'X']
emoticons = ["*O", "*-*", "*O*", "*o*", "* *", "=}","(:;)", "{;:]", "^_^", "^-^"]
for eye in eyes:
    for nose in noses:
        for mouth in mouths:
            emoticons.append(eye+nose+mouth)


eyes = [':', ';']
noses = ['', '-']
mouths = [')', 'D', ']', '>',  '(',  '[', '<', 'p', 'P', 'd']            
tongue_no_emo = []
for nose in noses:
    for eye in eyes:
        for mouth in mouths:
            tongue_no_emo.append(eye+nose+mouth)
    
class EmoticonExtractor(BaseEstimator, TransformerMixin):

    """ Model that extracts Char Based Features. Counter: 32 features """

    def __init__(self, emoticons, norm=True, norm_axis=0):
        self.emoticons = emoticons
        self.names = emoticons
        self.norm = norm
        self.norm_axis = norm_axis

    def fit(self, X, y=None):
        return self

    def transform(self, texts):
        """ transform data

        :texts: The texts to count capital words in
        :returns: array

        """
        from sklearn.preprocessing import normalize
        
        final_feat = []
        for text in texts:
            #text = text.encode('utf-8', 'ignore')
            tmp = [text.count(emo) for emo in self.emoticons]
            #if ';od' in text:
            #    print 'Kai omws'
            final_feat.append(tmp)
        final_feat2 = numpy.array(final_feat)
        final_feat3 = final_feat2[:, numpy.nonzero(final_feat2.sum(axis=0))[0]]
        print 'Indexes'
        print numpy.nonzero(final_feat2.sum(axis=0))[0]
        print type(numpy.nonzero(final_feat2.sum(axis=0))[0])
        print 'Names'
        print numpy.array(self.names).shape
        self.names = numpy.array(self.names)[numpy.nonzero(final_feat2.sum(axis=0))[0]]
        print final_feat3.shape
        if self.norm:
            final_feat3 = normalize(numpy.array(final_feat3), norm='l1', axis=0)
            return final_feat3
            #return normalize(numpy.array(final_feat), norm='l1', axis=self.norm_axis)
        else:
            return final_feat3

In [66]:
from sklearn.cross_validation import train_test_split
split = 0.2
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=split, stratify=y, random_state=100)
print len(X_train), len(X_cv), len(X_cv) + len(X_train), len(X)

348 88 436 436


In [72]:
aa = EmoticonExtractor(emoticons)
tr = aa.fit_transform(X_train)

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
tr.shape

In [ ]:
tr

In [ ]:
s = _twokenize.tokenizeRawTweetText('Lala this is funny! Lala! :D. :(')
aa = WordFeatureExtractor('./English_Function_Words_Set/')
tr = aa.transform(X_train)

In [ ]:
c = CharFeatureExtractor(True)
tr = c.fit_transform(X_train)

In [ ]:
tr = pipe3.steps[1][1].transform(pipe3.steps[0][1].transform(X_train))

In [ ]:
import pandas as pd
data = pd.DataFrame(pipe3.steps[1][1].term_table, columns=['soa_%d'% i for i in xrange(len(set(y)))])
#data["class"] = y_train
print(data.describe())

In [68]:
import pandas as pd
data = pd.DataFrame(tr, columns=aa.names)
data["class"] = y_train
print(data.describe())

               *O         * *         ^_^          :)          :D          :]  \
count  348.000000  348.000000  348.000000  348.000000  348.000000  348.000000   
mean     0.002874    0.002874    0.002874    0.002874    0.002874    0.002874   
std      0.028260    0.034671    0.031952    0.006732    0.014778    0.032748   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
50%      0.000000    0.000000    0.000000    0.000319    0.000000    0.000000   
75%      0.000000    0.000000    0.000000    0.002435    0.000000    0.000000   
max      0.400000    0.600000    0.500000    0.056691    0.175610    0.500000   

               :(          :[          :<          :p     ...             ;-3  \
count  348.000000  348.000000  348.000000  348.000000     ...      348.000000   
mean     0.002874    0.002874    0.002874    0.002874     ...        0.002874   
std      0.007759    0.0399

In [ ]:
from pandas.tools.plotting import scatter_matrix
scatter_matrix(data, alpha=0.2, figsize=(6, 6), diagonal='kde')
plt.show()

In [2]:
grouped = data.groupby('class')
means = grouped.mean().T

import matplotlib.pyplot as plt
### BAR PLOTS OF MEAN VALUE OF FEATURES FOR EACH CLASS ######

grouped = data.groupby('class')
#plt.figure()
grouped.sum().T.plot(kind='bar', figsize=(60,10), grid=True)
plt.savefig('test1.png')
plt.show()

NameError: name 'data' is not defined

In [90]:
def f(s):
    return s/float(sum(s))

print type(grouped)

aaa = grouped.sum()
aaa.loc['65-xx'].argmax()

<class 'pandas.core.groupby.DataFrameGroupBy'>


';P'

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from pan.features import SOA_Model2, SOAC_Model2
from pan import features

grams3 = TfidfVectorizer(analyzer='word', ngram_range=[1,1], max_features=3000)
soac = SOAC_Model2(max_df=0.9, min_df=5, tokenizer_var='sklearn', max_features=None)
combined = FeatureUnion([('3grams', grams3), ('soac', soac)])
svm = LinearSVC(C=100, class_weight='balanced')
#svm = SVC(kernel='rbf', C=100, gamma=1, class_weight='balanced', probability=True)
pipe = Pipeline([('combined',combined), ('svm', svm)])
pipe.steps



num_folds = 4
split = 0.2
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=split, stratify=y, random_state=100)
print len(X_train), len(X_cv), len(X_cv) + len(X_train), len(X)
#eclf = VotingClassifier(estimators=[("0", pipe1), ('1', pipe2)], voting='soft')
#pipe2.fit(X_train, y_train)
# trained_models = []
# params = {}
params = {'svm__C': [0.001, 0.01, 0.1, 1, 10, 100],
          'combined__soac__max_df': [1.0, 0.9, 0.8],
          'combined__soac__max_features': [None, 5000, 10000],
          'combined__soac__min_df': [1, 3, 5]}
for model in [pipe]:
    grid_search = GridSearchCV(estimator=model, param_grid=params, verbose=1, n_jobs=3, cv=num_folds, 
                                   refit=True, scoring='f1_weighted')
    grid_search.fit(X_train, y_train)
    print(grid_search.best_score_)
    #print(grid_search.best_estimator_) 
    print('Best_params')
    print(grid_search.grid_scores_)
best = grid_search.best_estimator_

In [33]:
print('Best_params')
print(grid_search.best_params_)

Best_params
{'combined__soac__min_df': 3, 'svm__C': 1, 'combined__soac__max_df': 1.0, 'combined__soac__max_features': 5000}


In [44]:
tr = aa.fit_transform(X_train)

ValueError: Found array with 0 feature(s) (shape=(242, 0)) while a minimum of 1 is required by the normalize function.

In [1]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline, FeatureUnion


aa =EmoticonExtractor(emoticons)
c = CharFeatureExtractor(norm_axis=0, norm=True)
svm2 = SVC(kernel='rbf', C=1, gamma=1, class_weight='balanced', probability=False)
#w = WordFeatureExtractor(function_words_path='./English_Function_Words_Set/', norm=True, norm_axis=0)
soacw = SOACWrapper()
svm = LinearSVC(C=0.001,  class_weight='balanced')
svm2 = SVC(kernel='linear', C=1, gamma=1, class_weight='balanced', probability=False)
pipe1 = Pipeline([('CharCount', aa), ('svm', svm2)])
#pipe2 = Pipeline([('WordCount', w), ('svm', svm2)])
pipe2 = Pipeline([('CharCount', c), ('soac', soacw),('svm', svm2)])

num_folds = 4
split = 0.2
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=split, stratify=y, random_state=100)
print len(X_train), len(X_cv), len(X_cv) + len(X_train), len(X)
#eclf = VotingClassifier(estimators=[("0", pipe1), ('1', pipe2)], voting='soft')
#pipe2.fit(X_train, y_train)
# trained_models = []
# params = {}
params = {'svm__C': [0.001, 0.01, 0.1, 1, 10, 100]}
for model in [pipe2]:
    grid_search = GridSearchCV(estimator=model, param_grid=params, verbose=1, n_jobs=2, cv=num_folds, 
                                   refit=True, scoring='f1_weighted')
    grid_search.fit(X_train,y_train)
    print(grid_search.best_score_)
    print(grid_search.best_estimator_) 
best = grid_search.best_estimator_

NameError: name 'EmoticonExtractor' is not defined

In [60]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
predict = best.predict(X_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y_cv))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y_cv))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

Accuracy : 0.475409836066
Confusion matrix :
 [[29  0]
 [32  0]]
Classification report :
              precision    recall  f1-score   support

     female       0.48      1.00      0.64        29
       male       0.00      0.00      0.00        32

avg / total       0.23      0.48      0.31        61



In [ ]:
import joblib
all_models = joblib.load("./")